### Onur Can
### Spectral Clustering
### 03.01.2022
#### 1. Data Import & Initial Data Plot ---------- 2. Connectivity Matrix -B- & Connectivity Plot
#### 3. Calculating D, L & L_symmetric ------  4. Find Smallest Eigen Vectors of L_Normalized &  Calculate Z
#### 5. K-means Application ---------------------- 6. Final Plot of Clusters & Centroids

In [1]:
# Onur Can 
# Project is done for Prof. Mehmet Gönen's DASC 521: Introduction to Machine Learning @ Koç University MSc Data Science Program
# Thanks Prof Mehmet for the dataset generation and instructions

import numpy as np                # For Array operations 
import matplotlib.pyplot as plt   # For Plotting
import scipy.spatial as spa       # For Euclidean distance calculations
import scipy.linalg as linalg     # For inverse Operations etc.

### 1. Data Import & Initial Data Plot

In [2]:
# Reading the data from given HW file.
X = np.genfromtxt("../input/em-cluster/EM_Clustering_data_set.csv", delimiter = ",")
N = X.shape[0]      # Data set size
D = X.shape[1]      # Dimensions
print(X.shape, N , D, X.dtype)

plt.figure(figsize = (6,6))
plt.plot(X[:,0], X[:,1], ".", markersize = 10, color = "black")
plt.xlabel("x1", fontsize = 12)
plt.ylabel("x2", fontsize = 12)
plt.title("Initial Data Visualization", fontsize = 16)
plt.show()

### 2. Connectivity Matrix -B- & Connectivity Plot

### 2.1. Connectivity Matrix -B-

In [3]:
threshold= +1.25                        # Treshold value to determine neighborhoods.
B_distances = spa.distance_matrix(X, X) # Euclidean distances of pairs , shape = (NxN)
print("------Pairwise Euclidean Distances-----\n\n",B_distances, B_distances.shape)
B = np.where((B_distances < threshold) & (B_distances != 0) , 1, 0) # A point cannot be neighbor of itself
print("\n------Connectivity Matrix (B)-----\n\n",B, B.shape)

### 2.2. Connectivity Plot

In [4]:
# Connectivity Plot
plt.figure(figsize = (6,6))
for j in range(N):
    connected_points = X[(B[j] == 1)]  # for every x points plot its neighbors in B matrix.
    for i in range(np.sum(B[j])):
        plt.plot([X[j,0], connected_points[i,0]], [X[j,1], connected_points[i,1]], "c-", linewidth = 0.4)
plt.plot(X[:,0], X[:,1], ".", markersize = 10, color = "black")
plt.xlabel("x1", fontsize = 12)
plt.ylabel("x2", fontsize = 12)
plt.title("Connectivity Matrix Plot", fontsize = 16)
plt.show()


### 3. Calculating D, L & L_symmetric
### 3.1. Active Connections Matrix -D-

In [5]:
diagonal = np.sum(B, axis = 1)  # sum of each row is calculated as an array
D = np.diag(diagonal)           # sum of each row put in the diag of D , shape = (NxN)
print("------ # of Active Connections Matrix (D)-----\n\n",D, D.shape)

### 3.2. Laplacian Matrix -L-

In [6]:
# L = D - B / Laplacian Matrix summarizes the incoming and outgoing links , shape = (NxN)
L = D - B
print("------ Laplacian Matrix (L)-----\n\n",L, L.shape)
print("Each row sum should be 0 : ", not np.all(np.sum(L, axis = 0))) # check each row sum is 0

### 3.3. L_symmetric

In [7]:
# L_symmetric is -normalized- version of L to prevent higher value domination ( Also : Random-Walk )
# L_symmetric = I - D^(-1/2) * B * D^(-1/2) , shape = (NxN)
I = np.eye((N))                                   # identity Matrix
D_modified = linalg.inv(D**0.5)                   # D^(-1/2)
L_symmetric = I - ( D_modified @ B @ D_modified )
print(L_symmetric, L_symmetric.shape)

### 4. Find Smallest Eigen Vectors of L_symmetric &  Calculate Z

In [8]:
# Parameters  given in the HW-file 
R = 5 # R-smallest eigen vectors of L ( dont pick smallest eigen value = 0)
K = 5 # Number of Clusters to be found in K-means

# Z = [v1 v2 v3 ..... v_r]  where v1 and v_r  = 1st and rth smallest eigen vectors , shape = (NxR)
L_eig_values, L_eig_vectors = linalg.eig(L_symmetric)  # eigen value-vector find
L_eig_values = np.real(L_eig_values)                   # only interested in real values
L_eig_vectors = np.real(L_eig_vectors)                 # only interested in real values

sorted_eig_values = np.argsort(L_eig_values)           #sort to find smallest
Z = L_eig_vectors[:, sorted_eig_values[1:R+1]]         # take corresponding eigen vectors
print(Z, Z.shape)


### 5. K-means Application
###  5.1. K-means Clustering Algorithm

In [9]:
# Inital centroids were changed according to HW File.
def update_centroids(memberships, X):
    if memberships is None:
    # initialize centroids
        centroids = np.array([Z[28,:], Z[142,:], Z[203,:], Z[270,:], Z[276,:]])  # HW_Row - 1 since python start 0
    else:
    # update centroids
        centroids = np.vstack([np.mean(X[memberships == k,:], axis = 0) for k in range(K)])
    return(centroids)

def update_memberships(centroids, X):
    # calculate distances between centroids and data points
    pair_dist = spa.distance_matrix(centroids, X)
    # find the nearest centroid for each data point
    memberships = np.argmin(pair_dist, axis = 0)
    return(memberships)

### 5.2. Visualization

In [10]:
def plot_current_state(centroids, memberships, X):
    cluster_colors = np.array(["#1f78b4", "#33a02c", "#e31a1c", "#ff7f00", "#6a3d9a", "#b15928",
                               "#a6cee3", "#b2df8a", "#fb9a99", "#fdbf6f", "#cab2d6", "#ffff99"])

    for c in range(K):
        plt.plot(X[memberships == c, 0], X[memberships == c, 1], ".", markersize = 10,
                 color = cluster_colors[c])
        plt.plot(centroids[c, 0], centroids[c, 1], "s", markersize = 12,
                 markerfacecolor = cluster_colors[c], markeredgecolor = "black")

    plt.xlabel("x1", fontsize = 12)
    plt.ylabel("x2", fontsize = 12)

### 5.3. Iterations

In [11]:
centroids = None
memberships = None
iteration = 1
while True:
    print("Iteration#{}:".format(iteration))
    if iteration == 100:  # just added to break if loop is infinite
        break

    old_centroids = centroids 
    centroids = update_centroids(memberships, Z)        
    # stopping condition if no parameter update
    if np.alltrue(centroids == old_centroids):
        break
    
    old_memberships = memberships
    memberships = update_memberships(centroids, Z)   
    # stopping condition if there are no membership updates
    if np.alltrue(memberships == old_memberships):
        plt.show()
        break

    iteration = iteration + 1

### 6. Final Plot of Clusters & Centroids

In [12]:
final_centroids = update_centroids(memberships, X) # update for centroids according to final members
plt.figure(figsize = (6,6))
plot_current_state(final_centroids,memberships,X)
plt.title("Final Clusters and Centroids", fontsize = 16)
plt.show()